In [27]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json

In [28]:
jsonl_path = "data/training_datasets/verified_nocomments.jsonl"
#jsonl_path = "data/training_datasets/verified_nocomments.jsonl"

df = pd.read_json(jsonl_path, lines=True)

In [29]:
df.head()

,filename,filetype,content,variant
0,cryptol/examples/splitAt.cry,cry,"x = [1,2,3,4] : [_][8]\n\ny = (splitAt x) : ([...",without_comments
1,cryptol/examples/AE.cry,cry,module AE where\n\nparameter\n type A : * ...,without_comments
2,cryptol/examples/xor_cipher.cry,cry,encrypt : {a}(fin a) => [8] -> [a][8] -> [a][8...,without_comments
3,cryptol/examples/zero_weird.cry,cry,x : {a}() => a -> [16]\nx v = zero v \n\nprope...,without_comments
4,cryptol/examples/builtin_lifting.cry,cry,"x = [True,False]\ny = [False,True]\n\nproperty...",without_comments


In [30]:
from preprocessing.interpreter_process import verify_df_row_with_cryptol
MOUNT_DIR = "/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files"
SERVER_URL = "http://localhost:8080"
print(verify_df_row_with_cryptol(df, 35, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL))
rows = []
for i in range(len(df)):
    if df.iloc[i]["filetype"] == 'cry':
        repl_result = verify_df_row_with_cryptol(
                df, i, host_mount_dir=MOUNT_DIR, server_url=SERVER_URL)
        rows.append({"filename" : df.iloc[i]["filename"], **repl_result["load_info"]})
    else:
        rows.append({"filename" : df.iloc[i]["filename"], "load_ok": False, "error": "saw", "file_deps": ["SAW"]})




{'host_path': '/Users/josh/SecurityAnalytics/DataPreprocess/cryptol-files/aes-mct-ecb_0h5ff8c_.cry', 'container_relpath': 'files/aes-mct-ecb_0h5ff8c_.cry', 'load_info': {'load_ok': True, 'file': 'files/aes-mct-ecb_0h5ff8c_.cry', 'error': 'None', 'file_deps': ['Cryptol', 'SuiteB']}}


In [31]:
repl_results_df = pd.DataFrame(rows)
repl_results_df.head()

,filename,load_ok,file,error,file_deps
0,cryptol/examples/splitAt.cry,True,files/splitAt_6_blcy4m.cry,None,[Cryptol]
1,cryptol/examples/AE.cry,True,files/AE_klwxs842.cry,None,error
2,cryptol/examples/xor_cipher.cry,True,files/xor_cipher_ahnd9_5_.cry,None,[Cryptol]
3,cryptol/examples/zero_weird.cry,True,files/zero_weird_ufcqcy0h.cry,None,[Cryptol]
4,cryptol/examples/builtin_lifting.cry,True,files/builtin_lifting_6rxbufyf.cry,None,[Cryptol]


In [32]:
repl_results_df.to_json(f"data/training_datasets/syntax_check/nocomments.jsonl", orient="records", lines=True)